In [1]:
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()

from bs4 import BeautifulSoup
from tqdm import tqdm
from multiprocessing_on_dill import Pool

In [2]:
async def fetch(session, id_):
    url = f'http://cartasocial.pt/resultados_pesquisadetalhe.php?&equip={id_}'
    async with session.get(url) as response:
        return await response.text()

def parse_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    info = [td.text.replace('\xa0', ' ').strip() for td in soup.findAll("td", {"class": "info2"})]
    info.append([td.text.replace('\xa0', ' ').strip().split('/') for td in soup.findAll("td", {"class": "info3"})])
    if info:
        return info
    else:
        return None

In [ ]:
ipss = []

asyncio.set_event_loop(asyncio.new_event_loop())
async with aiohttp.ClientSession() as session:
    aws = [fetch(session, id_) for id_ in range(10000)]
    for html in tqdm(asyncio.as_completed(aws), total=len(aws)):
        info = parse_html(await html)
        if info is not None:
            ipss.append(info)


100%|██████████| 10000/10000 [03:30<00:00, 47.45it/s]


In [6]:
pd.Series(ipss).apply(len).value_counts()

1    6436
6    3488
8      75
7       1
dtype: int64

In [8]:
pd.DataFrame(ipss).to_excel("dados ipss.xlsx")

In [ ]:
ipss_emails = pd.DataFrame(ipss)[2].values.tolist()

In [ ]:
ipss_emails = [e for e in ipss_emails if e]

In [ ]:
ipss_emails = [e for e in ipss_emails if '@' in e]

In [ ]:
len(ipss_emails)

In [ ]:
len(set(ipss_emails))

In [ ]:
pd.options.display.max_seq_items = 5000

In [ ]:
with open('emails.txt', 'w') as f:
    for item in set(ipss_emails):
        f.write("%s\n" % item)